In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Input, LeakyReLU, Layer, UpSampling2D,Add,Flatten,AveragePooling2D, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Load model Encoder 

In [3]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
encoder_model_path = '../VAE_model/model_VAE/encoder_model_256.h5'
decoder_model_path = '../VAE_model/model_VAE/decoder_model_256.h5'

In [5]:
model_encode = tf.keras.models.load_model(encoder_model_path, custom_objects={'Sampling': Sampling})

In [6]:
model_encode.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 1664        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 128)  204928      conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 256)  819456      conv2d_1[0][0]                   
_______________________________________________________________________________________

In [7]:
model_decode = tf.keras.models.load_model(decoder_model_path)

In [8]:
model_decode.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256)]             0         
_________________________________________________________________
dense (Dense)                (None, 65536)             16842752  
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 1024)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 512)       13107712  
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       3277056   
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 128)       819328    
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 128, 128, 64)     

# Load data fingerprint to encoder

In [10]:
DATA_DIR = "../Finger_enhancement/data_cropped/data_train/"

with open(os.path.join(DATA_DIR,"data_face_2856_train.pkl"), "rb") as input_file:
    data_train_face = pickle.load(input_file)
with open(os.path.join(DATA_DIR,"data_fingerprint_2856_train.pkl"), "rb") as input_file:
    data_train_finger = pickle.load(input_file)

    
with open(os.path.join(DATA_DIR,"data_face_2856_val.pkl"), "rb") as input_file:
    data_val_face = pickle.load(input_file)
with open(os.path.join(DATA_DIR,"data_fingerprint_2856_val.pkl"), "rb") as input_file:
    data_val_finger = pickle.load(input_file)

In [11]:
data_train_face.shape,data_train_finger.shape,data_val_face.shape,data_val_finger.shape

((2856, 256, 256, 3),
 (2856, 256, 256, 1),
 (715, 256, 256, 3),
 (715, 256, 256, 1))

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array

In [13]:
def augment_data(data_3d):
    num_data = data_3d.shape[0]
    num_data_augment = int(data_3d.shape[0]*0.5)
    index_data = np.random.choice(data_3d.shape[0],num_data_augment,replace=False)
    data_aug = []
    for i in index_data:
        data_aug.append(data_3d[i,:,:,:])
    samples = np.asarray(data_aug)    
    

    # create image data augmentation generator
    datagen = ImageDataGenerator(horizontal_flip=False,height_shift_range=0.1,width_shift_range=0.1)
    datagen.fit(samples)
    it = datagen.flow(samples,batch_size=1000,shuffle=False)
    results = np.concatenate([data_3d,it.next()],axis=0)
    np.random.shuffle(results)
    return results
def augment_data_face_fingerprint(data_face,data_finger, num_augment_percent=0.5):
    num_data = data_face.shape[0]
    num_data_augment = int(data_face.shape[0]*num_augment_percent)
    index_data = np.random.choice(data_face.shape[0],num_data_augment,replace=False)
    
    data_aug_face = []
    data_aug_finger = []
    for i in index_data:
        data_aug_face.append(data_face[i,:,:,:])
        data_aug_finger.append(data_finger[i,:,:,:])
    data_aug_face = np.asarray(data_aug_face)    
    data_aug_finger = np.asarray(data_aug_finger) 

    # create image data augmentation generator
    datagen_face = ImageDataGenerator(horizontal_flip=True)
    datagen_face.fit(data_aug_face)
    
    datagen_finger = ImageDataGenerator(horizontal_flip=False,height_shift_range=0.1,width_shift_range=0.1,shear_range=0.5,rotation_range=20)
    datagen_finger.fit(data_aug_finger)
    
    it_face = datagen_face.flow(data_aug_face,batch_size=num_data_augment,shuffle=False)
    it_finger = datagen_finger.flow(data_aug_finger,batch_size=num_data_augment,shuffle=False)
    
    results_face = np.concatenate([data_face,it_face.next()],axis=0)
    results_finger = np.concatenate([data_finger,it_finger.next()],axis=0)
    #np.random.shuffle(results)
    return results_face,results_finger

def add_noise_data(data_3d,noise_factor=1):
    num_data = data_3d.shape[0]
    num_data_add_noise = int(data_3d.shape[0]*0.3)
    index_data = np.random.choice(data_3d.shape[0],num_data_add_noise,replace=False)
    data_add_noise = []
    for i in index_data:
        data_add_noise.append(data_3d[i,:,:,:])
    samples = np.asarray(data_add_noise)
    data_none_noisy = np.asarray(data_add_noise)
    data_noisy = samples + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=samples.shape)  
    data_noisy = np.clip(data_noisy, 0., 1.)
    return data_noisy,data_none_noisy

In [14]:
data_train_face, data_train_finger = augment_data_face_fingerprint(data_train_face,data_train_finger)

In [15]:
data_train_face.shape,data_train_finger.shape,data_val_face.shape,data_val_finger.shape

((4284, 256, 256, 3),
 (4284, 256, 256, 1),
 (715, 256, 256, 3),
 (715, 256, 256, 1))

In [16]:
data_train_finger = data_train_finger.astype('float32')
data_train_finger = ((data_train_finger)/np.max(data_train_finger))
data_train_finger = np.where(data_train_finger > .5, 1.0, 0.0).astype('float32')
data_val_finger = data_val_finger.astype('float32')
data_val_finger = ((data_val_finger)/np.max(data_val_finger))
data_val_finger = np.where(data_val_finger > .5, 1.0, 0.0).astype('float32')

In [18]:
data_train_face = ((data_train_face-127.5)/127.5).astype('float32')
data_val_face = ((data_val_face-127.5)/127.5).astype('float32')

In [1]:
plt.subplot(1, 2, 1)
plt.imshow(data_train_finger[0],cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow((data_train_face[0].astype('uint8')))
plt.show()

# Get latent code

In [32]:
# mean, logvar,z = model_encode.predict(data_finger_val)
# result = [mean, logvar,z]

In [33]:
# result_finger = model_decode.predict(z)

In [34]:
# plt.imshow(data_finger_val[15])
# plt.show()
# plt.imshow(result_finger[15])
# plt.show()

In [35]:
# data_finger_val = resize_image(data_finger_val)

In [36]:
# data_finger_val.shape

In [37]:
# DIR_SAVE = "./data_train/data_split/"
# pickle.dump(result, open(os.path.join(DIR_SAVE,"latentcode_val_augment.pkl"),"wb"))
# pickle.dump(data_face_val, open(os.path.join(DIR_SAVE,"face_val_augment.pkl"),"wb"))
# pickle.dump(data_finger_val, open(os.path.join(DIR_SAVE,"finger_val_augment.pkl"),"wb"))

In [20]:
mean_train,logvar_train,z_train = model_encode.predict(data_train_finger)
mean_val,logvar_val,z_val = model_encode.predict(data_val_finger)

In [21]:
#result = model_decode.predict(z)

In [22]:
out_encoder_train = [mean_train,logvar_train,z_train]
out_encoder_val = [mean_val,logvar_val,z_val]

In [23]:
out_encoder_train = np.asarray(out_encoder_train)
out_encoder_val = np.asarray(out_encoder_val)

In [24]:
out_encoder_train.shape,out_encoder_val.shape

((3, 4284, 256), (3, 715, 256))

In [25]:
DIR_SAVE = "./data_train/data_get_latent/"
pickle.dump(out_encoder_train, open(os.path.join(DIR_SAVE,"data_latentcode_2856_train.pkl"),"wb"))
pickle.dump(out_encoder_val, open(os.path.join(DIR_SAVE,"data_latentcode_2856_val.pkl"),"wb"))

In [27]:
print(np.max(data_train_finger),np.median(data_train_finger),np.min(data_train_finger))
print(np.max(data_val_finger),np.median(data_val_finger),np.min(data_val_finger))

1.0 0.0 0.0
1.0 0.0 0.0


In [28]:
pickle.dump(data_train_finger, open(os.path.join(DIR_SAVE,"data_fingerprint_2856_train.pkl"),"wb"))
pickle.dump(data_train_face, open(os.path.join(DIR_SAVE,"data_face_2856_train.pkl"),"wb"))

pickle.dump(data_val_finger, open(os.path.join(DIR_SAVE,"data_fingerprint_2856_val.pkl"),"wb"))
pickle.dump(data_val_face, open(os.path.join(DIR_SAVE,"data_face_2856_val.pkl"),"wb"))